<a href="https://colab.research.google.com/github/PriyanshuChandela/SEO-Awareness/blob/main/Call_Quality_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yt-dlp pydub faster-whisper transformers
import yt_dlp
from pydub import AudioSegment

url = "https://www.youtube.com/watch?v=4ostqJD3Psc"

# Download audio
ydl_opts = {
    "format": "bestaudio/best",
    "outtmpl": "call_audio.%(ext)s"
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# Convert to wav (16k, mono)
audio = AudioSegment.from_file("call_audio.webm")
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("call_audio.wav", format="wav")

from faster_whisper import WhisperModel

model = WhisperModel("tiny.en")  # fast model for Colab
segments, info = model.transcribe("call_audio.wav")

transcript = []
for segment in segments:
    transcript.append({"start": segment.start, "end": segment.end, "text": segment.text})

print("Sample Transcript:", transcript[:5])


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading player 6742b2b9-main
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
[info] 4ostqJD3Psc: Downloading 1 format(s): 251
[download] Sleeping 2.00 seconds as required by the site...
[download] Destination: call_audio.webm
[download] 100% of    1.99MiB in 00:00:00 at 12.55MiB/s  


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Sample Transcript: [{'start': 0.0, 'end': 10.0, 'text': ' Thank you for calling Nissan, my name is Lauren.'}, {'start': 10.0, 'end': 11.0, 'text': ' Can I have your name?'}, {'start': 11.0, 'end': 14.0, 'text': ' Yeah, my name is John Smith.'}, {'start': 14.0, 'end': 15.0, 'text': ' Thank you, John.'}, {'start': 15.0, 'end': 16.0, 'text': ' How can I help you?'}]


In [3]:
import re
from transformers import pipeline

# 1. Talk-time ratio (approximation by words)
speaker_a, speaker_b = 0, 0
for i, seg in enumerate(transcript):
    if i % 2 == 0:
        speaker_a += len(seg["text"].split())
    else:
        speaker_b += len(seg["text"].split())

total = speaker_a + speaker_b
talk_ratio = {
    "Speaker A": round((speaker_a/total)*100, 2),
    "Speaker B": round((speaker_b/total)*100, 2)
}

# 2. Number of questions
all_text = " ".join([seg["text"] for seg in transcript])
questions = len(re.findall(r"\?", all_text))

# 3. Longest monologue
longest = max([seg["end"] - seg["start"] for seg in transcript])

# 4. Sentiment analysis
sentiment_analyzer = pipeline("sentiment-analysis")
sentiment = sentiment_analyzer(all_text[:512])[0]  # only first 512 chars (Colab limit)

# 5. Actionable Insight (simple rule-based)
insight = "Customer spoke less → Sales rep should ask more open-ended questions." \
    if talk_ratio["Speaker A"] > 70 else "Good balance, but focus on building rapport."

# Final report
report = {
    "Talk-time ratio": talk_ratio,
    "Questions asked": questions,
    "Longest monologue (sec)": round(longest, 2),
    "Overall Sentiment": sentiment,
    "Actionable Insight": insight
}

print(report)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


{'Talk-time ratio': {'Speaker A': 51.54, 'Speaker B': 48.46}, 'Questions asked': 8, 'Longest monologue (sec)': 10.0, 'Overall Sentiment': {'label': 'POSITIVE', 'score': 0.9985699653625488}, 'Actionable Insight': 'Good balance, but focus on building rapport.'}
